# Transfer Learning

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
batch_size = 32
img_size = 299
train_path = '../data/sports/train/'
test_path = '../data/sports/test/'

## Transfer learning

In [3]:
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [4]:
base_model = Xception(include_top=False,
                      weights='imagenet',
                      input_shape=(img_size, img_size, 3),
                      pooling='avg')

83689472/83683744 [==============================] - 82s 1us/step


In [5]:
model = Sequential([
    base_model,
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 2048)              20861480  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
Total params: 21,386,795
Trainable params: 21,332,267
Non-trainable params: 54,528
_________________________________________________________________


In [7]:
model.layers[0].trainable = False

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 2048)              20861480  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
Total params: 21,386,795
Trainable params: 525,315
Non-trainable params: 20,861,480
_________________________________________________________________


In [9]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
from keras.preprocessing.image import ImageDataGenerator

In [11]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

In [12]:
batch_size = 32

In [16]:
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    shuffle=False)

Found 2100 images belonging to 3 classes.


In [17]:
test_generator = datagen.flow_from_directory(
    test_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    shuffle=False)

Found 902 images belonging to 3 classes.


In [18]:
model.fit_generator(train_generator)

Epoch 1/1
66/66 [==============================] - 44s 661ms/step - loss: 1.2251 - acc: 0.3409


In [19]:
model.evaluate_generator(test_generator)

[0.8935164178818662, 0.45121951219512196]

Yay! in a single epoch we can classify images with a decent accuracy.

## Exercise

The base model takes an image and returns a vector of 2048 numbers. We will call this vector a bottleneck feature. Use the `base_model.predict_generator` function and the train generator to generate bottleneck features for the training set. Save these vectors to a numpy array using the code provided.

In [ ]:
base_model.summary()

In [ ]:
import os
os.makedirs('models', exist_ok = True)

In [ ]:
np.save(open('models/bf_train.npy', 'wb'), bf_train)

## Exercise 2

At https://keras.io/applications/#documentation-for-individual-models you can find all the pre-trained models available in keras.

- Try using another model, for example MobileNet and repeat the training. Do you get to a better accuracy?

check out:
- https://keras.io/applications/#usage-examples-for-image-classification-models
- https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
- https://keras.io/applications/